In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt   
HOME = os.environ["HOME"]
map_free_path = os.path.join(HOME, "map_free_localization/mapfree")

if os.path.exists(map_free_path):
    sys.path.append(map_free_path)

from torch.utils.data import DataLoader
from yacs.config import CfgNode as cfg
from lib.eval.sift_runner import SiftRunner
from lib.dataset.mapfree import MapFreeDataset
from lib.eval.eval import Eval

In [ ]:
data_dir = "../lib/tests/test_data"
config_path = os.path.join(data_dir, "testset.yaml")
config = cfg()
config.set_new_allowed(True)
config.DEBUG = False

if os.path.exists(config_path):
    config.merge_from_file(config_path)
    # explicitely setting to None because if loading from yaml it's a string
    config.DATASET.SCENES = None
    config.DATASET.AUGMENTATION_TYPE = None
    config.DATASET.DATA_ROOT = '/media/jprincen/HD/Map Free Localization'
    config.DATASET.DEPTH_ROOT = '/media/jprincen/HD/Map Free Localization/mickey_depths'
else:
    config = None
dataset = MapFreeDataset(config, "val")
dataloader = DataLoader(
    dataset,
    batch_size=1,
)
config_path = os.path.join("../config/sift", "sift_config.yaml")
sift_runner = SiftRunner(config_path)


In [ ]:
estimated_poses = sift_runner.run(dataloader)


In [ ]:
eval = Eval.fromMapFree(estimated_poses, dataset)

In [ ]:
np.median(eval.results['trans_err']), np.median(eval.results['rot_err']), np.median(eval.results['reproj_err'])

In [ ]:
reproj_err = eval.results['reproj_err']
_ = plt.hist(reproj_err)

In [ ]:
trans_err = eval.results['trans_err']
_ = plt.hist(trans_err)

In [ ]:
rot_err = eval.results['rot_err']
_ = plt.hist(rot_err)